# Code 4-28 to Code 4-51

In [1]:
import pandas as pd
from nltk.stem import WordNetLemmatizer 
import numpy as np
import nltk

Importing 2 datasets - one for creating the features (df_orig) and another one for identifying the "named entity" tags(t1)

In [2]:
t1 = pd.read_csv("ner.csv",encoding='latin1', error_bad_lines=False)
df_orig = pd.read_csv("ner_dataset.csv",encoding='latin1')

b'Skipping line 281837: expected 25 fields, saw 34\n'


In [3]:
df_orig.head(30)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [4]:
t1.head(10)

,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,...,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,...,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,...,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,...,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,...,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,...,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O
5,5,through,london,to,TO,lowercase,to,NNP,capitalized,London,...,have,VBP,lowercase,have,lowercase,marched,1.0,lowercase,through,O
6,6,london,to,protest,VB,lowercase,protest,TO,lowercase,to,...,march,VBN,lowercase,marched,lowercase,through,1.0,capitalized,London,B-geo
7,7,to,protest,the,DT,lowercase,the,VB,lowercase,protest,...,through,IN,lowercase,through,capitalized,London,1.0,lowercase,to,O
8,8,protest,the,war,NN,lowercase,war,DT,lowercase,the,...,london,NNP,capitalized,London,lowercase,to,1.0,lowercase,protest,O
9,9,the,war,in,IN,lowercase,in,NN,lowercase,war,...,to,TO,lowercase,to,lowercase,protest,1.0,lowercase,the,O


In [5]:
df_orig.columns = ["sentence_id","word","pos","tag"]
df_orig["sentence_id"] = df_orig["sentence_id"].fillna('none')
df_orig["tag"] = df_orig["tag"].fillna('none') 

In the below output - B and I represent the beginning or intermediate. We are interested in the ‘org’ tag wherever they are present

In [6]:
df_orig["tag"].value_counts()

O        887908
B-geo     37644
B-tim     20333
B-org     20143
I-per     17251
B-per     16990
I-org     16784
B-gpe     15870
I-geo      7414
I-tim      6528
B-art       402
B-eve       308
I-art       297
I-eve       253
B-nat       201
I-gpe       198
I-nat        51
Name: tag, dtype: int64

Below code reconstructs the sentence in df_orig and gets the POS tags of the word level features

In [7]:
lemmatizer = WordNetLemmatizer() 

list_sent = []
word_list = []
tag_list = []
for ind,row in df_orig[0:100000].iterrows():
    sid = row["sentence_id"]
    word = row["word"]
    tag = row["tag"]

    if((sid!="none") or (ind==0)):
        if(len(word_list)>0):
            list_sent.append(word_list)
            pos_tags_list = nltk.pos_tag(word_list)
            df = pd.DataFrame(pos_tags_list)
            df["id"] = sid_perm
            
            try:
                df["tag"] = tag_list
            except:
                print (tag_list,word_list,len(tag_list),len(word_list)) 
            if(sid_perm=="Sentence: 1"):
                df_all_pos = df
                
            else:
                df_all_pos = pd.concat([df_all_pos,df],axis=0)
            
        word_list = []
        tag_list = []
        word_list.append(word)
        tag_list.append(tag)
    else:
         word_list.append(word)
        #word_list = word_list + word + " "
         tag_list.append(tag)
        
    if(sid!="none"):
        sid_perm = sid


In [8]:
df_all_pos.shape

(99998, 4)

In [9]:
df_all_pos.head()

,0,1,id,tag
0,Thousands,NNS,Sentence: 1,O
1,of,IN,Sentence: 1,O
2,demonstrators,NNS,Sentence: 1,O
3,have,VBP,Sentence: 1,O
4,marched,VBN,Sentence: 1,O


In [10]:
###think space is creating issues

In [11]:
df_all_pos.columns = ["word","pos_tag","sid","tag"]

df_all is the data frame that contains the word and “POS” tags. Once this step is done we could get lemma of the word, shape of the word, length etc which are not dependent on the sentence but just the words themselves

We would also want to identify words that have either numbers or special characters in them. We would tag these words into a seperate word type before replacing the special characters for further processing


In [12]:
df_all_pos["word1"] = df_all_pos["word"] 

df_all_pos["word1"] = df_all_pos.word1.str.replace('[^a-z\s]+','') 

df_all_pos["word_type"] = "normal"
df_all_pos.loc[df_all_pos.word.str.contains('[0-9]+'),"word_type"] = "number"
df_all_pos.loc[df_all_pos.word.str.contains('[^a-zA-Z\s]+'),"word_type"] = "special_chars"

df_all_pos.loc[(df_all_pos.word_type!="normal") & (df_all_pos.word1.str.len()==0),"word_type"] = "only_special"


In [13]:
def lemma_func(x):
    return lemmatizer.lemmatize(x)

In [14]:
df_all_pos["shape"] = "mixed"
df_all_pos.loc[((df_all_pos.word.str.islower()==True) & (df_all_pos.word_type=="normal")),"shape"]="lower"
df_all_pos.loc[((df_all_pos.word.str.islower()==False) & (df_all_pos.word_type=="normal")),"shape"]="upper"

df_all_pos["lemma"] = df_all_pos["word"].apply(lemma_func)

df_all_pos["length"] = df_all_pos["word"].str.len()

Getting the “relative position” of the word in the sentences and the sentence length of the sentence

In [15]:
df_all_pos["ind_num"] = df_all_pos.index
df_all_pos["sent_len"]=df_all_pos.groupby(["sid"])["ind_num"].transform(max)
df_all_pos1 = df_all_pos[df_all_pos.sent_len>0]
df_all_pos1["rel_position"] = df_all_pos1["ind_num"] /df_all_pos1["sent_len"]*100
df_all_pos1["rel_position"] = df_all_pos1["rel_position"].astype('int')

In [16]:
df_all_pos1.columns

Index(['word', 'pos_tag', 'sid', 'tag', 'word1', 'word_type', 'shape', 'lemma',
       'length', 'ind_num', 'sent_len', 'rel_position'],
      dtype='object')

we use the pandas “shift” functionality to get features like previous word, next word, POS tag of the previous word, POS tag of the next tag etc. For each feature we call the get_prev_next to get the shifted features

In [17]:
def get_prev_next(df_all_pos,col_imp):
    prev_col = "prev_" + col_imp
    next_col = col_imp + "_next"
    
    prev_col1 = "prev_prev_" + col_imp
    next_col1 = col_imp + "_next_next"
    
    df_all_pos[prev_col] = df_all_pos[col_imp].shift(1)
    df_all_pos.loc[df_all_pos.index==0,prev_col] = "start"
    
    df_all_pos[next_col] = df_all_pos[col_imp].shift(-1)
    df_all_pos.loc[df_all_pos.index==df_all_pos.sent_len,next_col] = "end"
    
    df_all_pos[prev_col1] = df_all_pos[col_imp].shift(2)
    df_all_pos.loc[df_all_pos.index<2,prev_col1] = "start"
    
    df_all_pos[next_col1] = df_all_pos[col_imp].shift(-2)
    df_all_pos.loc[(df_all_pos.sent_len-df_all_pos.index)<=1,next_col1] = "end"
    
    return df_all_pos

In [18]:
df_all_pos1 = get_prev_next(df_all_pos1,"word")
df_all_pos1 = get_prev_next(df_all_pos1,"lemma")
df_all_pos1 = get_prev_next(df_all_pos1,"shape")
df_all_pos1 = get_prev_next(df_all_pos1,"pos_tag")
df_all_pos1 = get_prev_next(df_all_pos1,"word_type")

In [19]:
df_all_pos1.columns

Index(['word', 'pos_tag', 'sid', 'tag', 'word1', 'word_type', 'shape', 'lemma',
       'length', 'ind_num', 'sent_len', 'rel_position', 'prev_word',
       'word_next', 'prev_prev_word', 'word_next_next', 'prev_lemma',
       'lemma_next', 'prev_prev_lemma', 'lemma_next_next', 'prev_shape',
       'shape_next', 'prev_prev_shape', 'shape_next_next', 'prev_pos_tag',
       'pos_tag_next', 'prev_prev_pos_tag', 'pos_tag_next_next',
       'prev_word_type', 'word_type_next', 'prev_prev_word_type',
       'word_type_next_next'],
      dtype='object')

Creating a binary "org/no_org" for the dependent variable

In [20]:
t2 = df_all_pos1[df_all_pos1.tag.isna()==False]
t2["tag1"] = "no_org"
t2.loc[t2.tag.str.contains('org',case=False),"tag1"]="org"

In [25]:
t2["tag"].value_counts()

O        84723
B-geo     3303
B-org     1876
I-per     1846
B-tim     1823
B-gpe     1740
B-per     1668
I-org     1470
I-geo      690
I-tim      549
B-art       75
B-eve       53
I-gpe       51
I-eve       47
I-art       43
B-nat       30
I-nat       11
Name: tag, dtype: int64

Creating a higher sample of "org" tags in order for the model to learn

In [27]:
t2_neg = t2.loc[t2.tag1!="org",:]
t2_neg1 = t2_neg.sample(frac=0.1)

t2_pos = t2.loc[t2.tag1=="org",:]
t3 = pd.concat([t2_neg1,t2_pos],axis=0)
t3 = t3.reset_index()
len(t2_neg),len(t2_neg1),len(t2_pos),len(t3)

(96652, 9665, 3346, 13011)

Stratified Split for train and test

In [28]:
tgt = t3["tag1"]

In [29]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(test_size=0.2,random_state=42,n_splits=1)

for train_index, test_index in sss.split(t3, tgt):
    x_train, x_test = t3[t3.index.isin(train_index)], t3[t3.index.isin(test_index)]
    y_train, y_test = t3.loc[t3.index.isin(train_index),"tag1"], t3.loc[t3.index.isin(test_index),"tag1"]


We would be also separating the string and numeric columns. The categorical columns will get an embedding layer and the numeric columns will get added to the flattened layers of the embedding output


In [30]:
num_cols = ['length','sent_len', 'rel_position']

str_cols = ['word', 'pos_tag', 'shape', 'lemma', 'word_type',
        'prev_word', 'word_next', 'prev_prev_word',
       'word_next_next', 'prev_lemma', 'lemma_next', 'prev_prev_lemma',
       'lemma_next_next', 'prev_shape', 'shape_next', 'prev_prev_shape',
       'shape_next_next', 'prev_pos_tag', 'pos_tag_next', 'prev_prev_pos_tag',
       'pos_tag_next_next','prev_word_type', 'word_type_next', 'prev_prev_word_type',
       'word_type_next_next']

x_train_num = x_train[num_cols]
x_test_num = x_test[num_cols]


Tokenization and label encoding: Input to the embedding layer is a label encoded categorical variables (“texts_to_sequences”). Once encoded this also needs to be padded with the required length. Since we have only single tokens per column we are keeping the max_len to 1. The tokenization gets the mapping words to indexes from the train and this is applied to text. Any unknown text in the test set will be ignored




In [31]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()

def conv_str_cols(col_tr,col_te):
    #print(col_tr)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(col_tr)
    
    col_tr1 = tokenizer.texts_to_sequences(col_tr)
    col_te1 = tokenizer.texts_to_sequences(col_te)
    col_tr2 = pad_sequences(col_tr1, maxlen=1, dtype='int32', padding='pre')
    col_te2 = pad_sequences(col_te1, maxlen=1, dtype='int32', padding='pre')
    
    return col_tr2,col_te2,tokenizer

Using TensorFlow backend.


In [32]:
tkn_list = []
for num,i in enumerate(str_cols):
    
    var1,var2,tkn= conv_str_cols(x_train[i],x_test[i])
    tkn_list.append(tkn)
    print (var1.shape,var2.shape)
    if(num==0):
    
        var_all_train = var1
        var_all_test = var2
       
    else:
        var_all_train = np.concatenate([var_all_train,var1],axis=1)
        var_all_test = np.concatenate([var_all_test,var2],axis=1)


(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)
(10408, 1) (2603, 1)


Below code does the following

1.	Creates the embeddings for each of the categorical variable and flattens them
2.	Appends all of the independent inputs into a list (inputs)
3.	Appends the flattened outputs into another list called “dense” list object (outputs)
4.	Creates separate numeric input and appends it to the “dense” list object
5.	Connects the numeric input to “num_inp1” to the “dense” list object
6.	Concatenates all the embeddings and the numeric input to 1 layer
7.	df_list is the list of all categorical and numeric train inputs
8.	df_list_test is the list of all categorical and numeric test inputs


In [33]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten,Concatenate
from keras.layers.merge import concatenate
from keras.layers import Input, Dense, Dropout, Flatten
from keras.models import Model
from keras.utils import to_categorical
from keras.optimizers import Adam,SGD
from keras.layers import Embedding
from keras.layers import Reshape
from keras.layers import add

In [34]:
embed_size=0
models = []
inputs = []
outputs = []
dense = []
df_list = []
df_list_test=[]
for num,categoical_var in enumerate(range(var_all_train.shape[1]) ):


    model = Sequential()
    no_of_unique_cat =np.max(var_all_train[:,num]) + 1
    
    embedding_size = min(np.ceil((no_of_unique_cat)/2), 50 )
    embedding_size = int(embedding_size)
    embed_size = embed_size + embedding_size
    vocab  = no_of_unique_cat+1
    
    
    A1 = Input(shape=(1,))
    A2 = Embedding(vocab,embedding_size,input_length = 1)(A1)
    
    A3 = Flatten()(A2)
    dense.append(A3)
    inputs.append(A1)
    
       
    df_list.append(var_all_train[:,num])
    df_list_test.append(var_all_test[:,num])

    
num_shape = x_train_num.shape[1]
embed_size = embed_size + num_shape
num_inp = Input(shape=(num_shape,))
num_inp1 = Dense(num_shape,activation = 'relu')(num_inp)
dense.append(num_inp1)
inputs.append(num_inp)

st_size = int(embed_size/2)

df_list.append(x_train_num)
df_list_test.append(x_test_num)

merge_one = concatenate(dense)   


Instructions for updating:
Colocations handled automatically by placer.


The layer merge_one will now be connected to other dense layers. I am going to reduce the number of nodes in each of the following layers by 2 and obtain a list of layer parameters. “layers_list” contains the architecture of the neural network after the merge_one. The function get_nn_mod lays out the rest of the architecture from merge_one based on the “layers_list”.  “Model” is a function which finally takes the initial inputs and the final outputs. The “model” object is now ready to be compiled

In [35]:
####flat layers list
layers_list = []
st_size1 = st_size
while (st_size1>10):
    st_size1 = int(st_size1/2)
    layers_list.append(st_size1)

layers_list

[153, 76, 38, 19, 9]

In [36]:
def get_nn_mod(list_layers,input1,dp,inputs):
    layers = []

    for num,i in enumerate(list_layers):
        print(num,i)
        
        if(num==0):
            input_orig = input1
 
            
        else:
            input1 = Dense(i, activation='relu')(input1)
            input1 = Dropout(dp)(input1)
     
        
    
    input_last = Dense(2, activation='softmax')(input1)

    model = Model(inputs=inputs, outputs=input_last)

    opt = SGD(lr=0.01, clipnorm=1.)
# Compile model
    model.compile(optimizer=opt, loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [37]:
final_model = get_nn_mod(layers_list,merge_one,0.6,inputs)

0 153
1 76
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2 38
3 19
4 9


In [38]:
final_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_5 (I

Converting the dependant variable to categorical encoding

In [39]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train1 = le.fit_transform(y_train)
y_test1 = le.fit_transform(y_test)

y_train2 = to_categorical(y_train1)
y_test2 = to_categorical(y_test1)
y_train2

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [40]:
# You could tweak this by adding class_weight example :class_weight = {0:8,1:6}

In [41]:


final_model.fit(df_list,y_train2, batch_size=10, epochs=30,
          verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
 - 13s - loss: 0.6911 - acc: 0.6959
Epoch 2/30
 - 9s - loss: 0.5837 - acc: 0.7416
Epoch 3/30
 - 8s - loss: 0.5762 - acc: 0.7427
Epoch 4/30
 - 9s - loss: 0.5737 - acc: 0.7424
Epoch 5/30
 - 9s - loss: 0.5799 - acc: 0.7428
Epoch 6/30
 - 9s - loss: 0.5642 - acc: 0.7426
Epoch 7/30
 - 9s - loss: 0.5402 - acc: 0.7427
Epoch 8/30
 - 9s - loss: 0.4811 - acc: 0.7427
Epoch 9/30
 - 8s - loss: 0.4500 - acc: 0.7422
Epoch 10/30
 - 8s - loss: 0.4487 - acc: 0.7983
Epoch 11/30
 - 8s - loss: 0.4208 - acc: 0.8413
Epoch 12/30
 - 9s - loss: 0.4297 - acc: 0.8395
Epoch 13/30
 - 9s - loss: 0.4372 - acc: 0.8466
Epoch 14/30
 - 8s - loss: 0.4589 - acc: 0.8486
Epoch 15/30
 - 9s - loss: 0.4370 - acc: 0.8554
Epoch 16/30
 - 9s - loss: 0.4275 - acc: 0.8592
Epoch 17/30
 - 9s - loss: 0.4250 - acc: 0.8641
Epoch 18/30
 - 8s - loss: 0.4242 - acc: 0.8692
Epoch 19/30
 - 8s - loss: 0.4421 - acc: 0.8705
Epoch 20/30
 - 9s - loss: 0.4336 - acc: 0.8732
Epoch 21/30
 - 9s - 

In [52]:
len(df_list_test),len(df_list)

(26, 26)

Inspecting the predicted outout

In [53]:
pred1=final_model.predict(df_list_test)
pred = pred1.argmax(axis=-1)

In [54]:
pd.Series(pred).value_counts()

0    1840
1     763
dtype: int64

In [55]:
pd.Series(pred).value_counts()

0    1840
1     763
dtype: int64

Measuring Accuracy

In [56]:

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
ac1 = accuracy_score(y_test1, pred)
print (ac1, f1_score(y_test1, pred, average='macro'))

0.9093353822512485 0.8863312046848031


In [57]:
rows_name = [0,1]

In [58]:
from sklearn.metrics import confusion_matrix

#pred_inv = le.inverse_transform(pred)

cmat = pd.DataFrame(confusion_matrix(y_test1, pred, labels=[0,1], sample_weight=None))
cmat.columns = rows_name 
cmat["act"] = rows_name
cmat

,0,1,act
0,1769,165,0
1,71,598,1


Save the model and tokenizer for doing the model execution

In [55]:
import pickle
def pik_now(ob_name):
    fl_out1 = fl_out + "/" + ob_name
    pickling_on = open(fl_out1,"wb")
    pickle.dump(eval(ob_name), pickling_on)
    pickling_on.close()
    
    return

In [56]:
fl_out =  "model_punc"
pikl_list = ["tkn_list"]

In [57]:
for i in pikl_list:
    pik_now(i)

In [58]:
from keras.models import load_model

# Creates a HDF5 file 'my_model.h5'
final_model.save('model_punc/model_ner.h5')

